In [2]:
import os
import argparse
import time
import numpy as np

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim

In [4]:
parser = argparse.ArgumentParser('ODE demo')
parser.add_argument('--method', type=str, choices=['dopri5', 'adams'], default='dopri5')
parser.add_argument('--data_size', type=int, default=1000)
parser.add_argument('--batch_time', type=int, default=10)
parser.add_argument('--batch_size', type=int, default=20)
parser.add_argument('--niters', type=int, default=500)
parser.add_argument('--test_freq', type=int, default=20)
parser.add_argument('--viz', action='store_true')
parser.add_argument('--gpu', type=int, default=0)
parser.add_argument('--adjoint', action='store_true')
# line below is edited to fix error
args, unknown = parser.parse_known_args()

In [5]:
if args.adjoint:
    from torchdiffeq import odeint_adjoint as odeint
else:
    from torchdiffeq import odeint

In [6]:
device = torch.device('cuda:' + str(args.gpu) if torch.cuda.is_available() else 'cpu')

In [7]:
true_y0 = torch.tensor([[2., 0.]]).to(device)
t = torch.linspace(0., 25., args.data_size).to(device)
true_A = torch.tensor([[-0.1, 2.0], [-2.0, -0.1]]).to(device)
print(t.shape)

torch.Size([1000])


In [8]:
class Lambda(nn.Module):

    def forward(self, t, y):
        return torch.mm(y**3, true_A)

In [9]:
with torch.no_grad():
    true_y = odeint(Lambda(), true_y0, t, method='dopri5')
print(true_y.shape)
print(true_y)

torch.Size([1000, 1, 2])
tensor([[[ 2.0000,  0.0000]],

        [[ 1.9795,  0.3944]],

        [[ 1.9494,  0.7742]],

        ...,

        [[-0.4418,  0.2882]],

        [[-0.4427,  0.2839]],

        [[-0.4436,  0.2794]]], device='cuda:0')


In [10]:
def get_batch():
    s = torch.from_numpy(np.random.choice(np.arange(args.data_size - args.batch_time, dtype=np.int64), args.batch_size, replace=False))
    batch_y0 = true_y[s]  # (M, D)
    batch_t = t[:args.batch_time]  # (T)
    batch_y = torch.stack([true_y[s + i] for i in range(args.batch_time)], dim=0)  # (T, M, D)
    return batch_y0.to(device), batch_t.to(device), batch_y.to(device)

In [11]:
by0, bt, by = get_batch()
print(by0.shape)
print(bt.shape)
print(by.shape)

torch.Size([20, 1, 2])
torch.Size([10])
torch.Size([10, 20, 1, 2])


In [34]:
def makedirs(dirname):
    if not os.path.exists(dirname):
        os.makedirs(dirname)

In [35]:
if args.viz:
    makedirs('png')
    import matplotlib.pyplot as plt
    fig = plt.figure(figsize=(12, 4), facecolor='white')
    ax_traj = fig.add_subplot(131, frameon=False)
    ax_phase = fig.add_subplot(132, frameon=False)
    ax_vecfield = fig.add_subplot(133, frameon=False)
    plt.show(block=False)

In [36]:
def visualize(true_y, pred_y, odefunc, itr):

    if args.viz:

        ax_traj.cla()
        ax_traj.set_title('Trajectories')
        ax_traj.set_xlabel('t')
        ax_traj.set_ylabel('x,y')
        ax_traj.plot(t.cpu().numpy(), true_y.cpu().numpy()[:, 0, 0], t.cpu().numpy(), true_y.cpu().numpy()[:, 0, 1], 'g-')
        ax_traj.plot(t.cpu().numpy(), pred_y.cpu().numpy()[:, 0, 0], '--', t.cpu().numpy(), pred_y.cpu().numpy()[:, 0, 1], 'b--')
        ax_traj.set_xlim(t.cpu().min(), t.cpu().max())
        ax_traj.set_ylim(-2, 2)
        ax_traj.legend()

        ax_phase.cla()
        ax_phase.set_title('Phase Portrait')
        ax_phase.set_xlabel('x')
        ax_phase.set_ylabel('y')
        ax_phase.plot(true_y.cpu().numpy()[:, 0, 0], true_y.cpu().numpy()[:, 0, 1], 'g-')
        ax_phase.plot(pred_y.cpu().numpy()[:, 0, 0], pred_y.cpu().numpy()[:, 0, 1], 'b--')
        ax_phase.set_xlim(-2, 2)
        ax_phase.set_ylim(-2, 2)

        ax_vecfield.cla()
        ax_vecfield.set_title('Learned Vector Field')
        ax_vecfield.set_xlabel('x')
        ax_vecfield.set_ylabel('y')

        y, x = np.mgrid[-2:2:21j, -2:2:21j]
        dydt = odefunc(0, torch.Tensor(np.stack([x, y], -1).reshape(21 * 21, 2)).to(device)).cpu().detach().numpy()
        mag = np.sqrt(dydt[:, 0]**2 + dydt[:, 1]**2).reshape(-1, 1)
        dydt = (dydt / mag)
        dydt = dydt.reshape(21, 21, 2)

        ax_vecfield.streamplot(x, y, dydt[:, :, 0], dydt[:, :, 1], color="black")
        ax_vecfield.set_xlim(-2, 2)
        ax_vecfield.set_ylim(-2, 2)

        fig.tight_layout()
        plt.savefig('png/{:03d}'.format(itr))
        plt.draw()
        plt.pause(0.001)

In [37]:
class ODEFunc(nn.Module):

    def __init__(self):
        super(ODEFunc, self).__init__()

        self.net = nn.Sequential(
            nn.Linear(2, 50),
            nn.Tanh(),
            nn.Linear(50, 2),
        )

        for m in self.net.modules():
            if isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, mean=0, std=0.1)
                nn.init.constant_(m.bias, val=0)

    def forward(self, t, y):
        return self.net(y**3)

In [38]:
class RunningAverageMeter(object):
    """Computes and stores the average and current value"""

    def __init__(self, momentum=0.99):
        self.momentum = momentum
        self.reset()

    def reset(self):
        self.val = None
        self.avg = 0

    def update(self, val):
        if self.val is None:
            self.avg = val
        else:
            self.avg = self.avg * self.momentum + val * (1 - self.momentum)
        self.val = val


In [39]:
ii = 0

In [40]:
func = ODEFunc().to(device)

In [41]:
optimizer = optim.RMSprop(func.parameters(), lr=1e-3)
end = time.time()

In [42]:
time_meter = RunningAverageMeter(0.97)

In [43]:
loss_meter = RunningAverageMeter(0.97)

In [44]:
batch_y0, batch_t, batch_y = get_batch()
print(batch_y0.shape)
print(batch_t.shape)

torch.Size([20, 1, 2])
torch.Size([10])


In [45]:
for itr in range(1, args.niters + 1):
        optimizer.zero_grad()
        batch_y0, batch_t, batch_y = get_batch()
        pred_y = odeint(func, batch_y0, batch_t).to(device)
        loss = torch.mean(torch.abs(pred_y - batch_y))
        loss.backward()
        optimizer.step()

        time_meter.update(time.time() - end)
        loss_meter.update(loss.item())

        if itr % args.test_freq == 0:
            with torch.no_grad():
                pred_y = odeint(func, true_y0, t)
                loss = torch.mean(torch.abs(pred_y - true_y))
                print('Iter {:04d} | Total Loss {:.6f}'.format(itr, loss.item()))
                visualize(true_y, pred_y, func, ii)
                ii += 1
        end = time.time()

Iter 0020 | Total Loss 0.704405
Iter 0040 | Total Loss 0.646594
Iter 0060 | Total Loss 0.896133
Iter 0080 | Total Loss 0.916797
Iter 0100 | Total Loss 0.786801
Iter 0120 | Total Loss 0.606235
Iter 0140 | Total Loss 0.504052
Iter 0160 | Total Loss 0.812353
Iter 0180 | Total Loss 0.347902
Iter 0200 | Total Loss 0.402888
Iter 0220 | Total Loss 0.538116
Iter 0240 | Total Loss 0.338865
Iter 0260 | Total Loss 0.314633
Iter 0280 | Total Loss 0.306688
Iter 0300 | Total Loss 0.635846
Iter 0320 | Total Loss 0.666869
Iter 0340 | Total Loss 0.603634
Iter 0360 | Total Loss 0.651890
Iter 0380 | Total Loss 0.280985
Iter 0400 | Total Loss 0.353793
Iter 0420 | Total Loss 0.423785
Iter 0440 | Total Loss 0.384070
Iter 0460 | Total Loss 0.410169
Iter 0480 | Total Loss 0.266400
Iter 0500 | Total Loss 0.488250


In [46]:
print(pred_y.shape)
print(true_y.shape)
print(true_y0.shape)

torch.Size([1000, 1, 2])
torch.Size([1000, 1, 2])
torch.Size([1, 2])
